## Задание #1 - Обработка логов.

In [1]:
import pandas as pd 

In [2]:
file_path = './log.csv'
df = pd.read_csv(file_path, parse_dates=['date']) # Сгенерировал маленький лог для примера.

In [3]:
df.dtypes

trace             int64
state            object
date     datetime64[ns]
dtype: object

### Merge для нахождения всех возможных ребер. Далее фильтрация, т.к. получаются дубликаты ребер с противоположным направлением.

In [4]:
df_from_to = df.merge(df, on='trace', how='inner', suffixes=('1', '2'))
df_from_to = df_from_to[df_from_to.date1 < df_from_to.date2]
df_from_to.head(10)

,trace,state1,date1,state2,date2
1,1,state_1,2025-01-01 10:00:00,state_2,2025-01-01 10:10:00
2,1,state_1,2025-01-01 10:00:00,state_3,2025-01-01 10:20:00
5,1,state_2,2025-01-01 10:10:00,state_3,2025-01-01 10:20:00
10,2,state_1,2025-01-01 10:05:00,state_2,2025-01-01 10:15:00
11,2,state_1,2025-01-01 10:05:00,state_2,2025-01-01 10:17:00
12,2,state_1,2025-01-01 10:05:00,state_3,2025-01-01 10:25:00
15,2,state_2,2025-01-01 10:15:00,state_2,2025-01-01 10:17:00
16,2,state_2,2025-01-01 10:15:00,state_3,2025-01-01 10:25:00
20,2,state_2,2025-01-01 10:17:00,state_3,2025-01-01 10:25:00
26,3,state_1,2025-01-01 11:00:00,state_2,2025-01-01 11:10:00


### Нахождение разницы во времени для последующей агрегации.

In [5]:
df_from_to['time_diff'] = df_from_to['date2'] - df_from_to['date1']
df_from_to.head(10)

,trace,state1,date1,state2,date2,time_diff
1,1,state_1,2025-01-01 10:00:00,state_2,2025-01-01 10:10:00,0 days 00:10:00
2,1,state_1,2025-01-01 10:00:00,state_3,2025-01-01 10:20:00,0 days 00:20:00
5,1,state_2,2025-01-01 10:10:00,state_3,2025-01-01 10:20:00,0 days 00:10:00
10,2,state_1,2025-01-01 10:05:00,state_2,2025-01-01 10:15:00,0 days 00:10:00
11,2,state_1,2025-01-01 10:05:00,state_2,2025-01-01 10:17:00,0 days 00:12:00
12,2,state_1,2025-01-01 10:05:00,state_3,2025-01-01 10:25:00,0 days 00:20:00
15,2,state_2,2025-01-01 10:15:00,state_2,2025-01-01 10:17:00,0 days 00:02:00
16,2,state_2,2025-01-01 10:15:00,state_3,2025-01-01 10:25:00,0 days 00:10:00
20,2,state_2,2025-01-01 10:17:00,state_3,2025-01-01 10:25:00,0 days 00:08:00
26,3,state_1,2025-01-01 11:00:00,state_2,2025-01-01 11:10:00,0 days 00:10:00


### Агрегация по:
1)      количество проходов по ребру (всего)
2)      количество уникальных trace, которые прошли по этому ребру
3)      min,max, avg по времени для каждого ребра

In [6]:
result = df_from_to.groupby(['state1', 'state2']).agg(
    count_all=('trace', 'count'), 
    count_unique=('trace', pd.Series.nunique),
    dt_min=('time_diff', 'min'),  
    dt_max=('time_diff', 'max'),  
    dt_delta=('time_diff', 'mean') 
).reset_index()

In [7]:
result

,state1,state2,count_all,count_unique,dt_min,dt_max,dt_delta
0,state_1,state_2,6,4,0 days 00:05:00,0 days 00:20:00,0 days 00:11:10
1,state_1,state_3,4,4,0 days 00:15:00,0 days 00:30:00,0 days 00:21:15
2,state_2,state_2,2,2,0 days 00:02:00,0 days 00:15:00,0 days 00:08:30
3,state_2,state_3,6,4,0 days 00:05:00,0 days 00:25:00,0 days 00:11:20
